# БПМ-19-3, Набиев Илкин, вариант 57

In [1]:
!pip install crc64iso

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from typing import List
from math import log2, ceil
from random import randrange
from crc64iso.crc64iso import crc64

In [3]:
def __hamming_common(src: List[List[int]], s_num: int, encode=True) -> int:
    s_range = range(s_num)
    errors = 0
    for i in src:
        sindrome = 0
        for s in s_range:
            sind = 0
            for p in range(2 ** s, len(i) + 1, 2 ** (s + 1)):
                for j in range(2 ** s):
                    if (p + j) > len(i):
                        break
                    sind ^= i[p + j - 1]
            if encode:
                i[2 ** s - 1] = sind
            else:
                sindrome += (2 ** s * sind)
        if (not encode) and sindrome:
            try:
                i[sindrome - 1] = int(not i[sindrome - 1])
            except IndexError:
                errors += 1
    return errors

In [4]:
def hamming_encode(msg: str, mode: int = 8) -> str:
    """
    Encoding the message with Hamming code.
    :param msg: Message string to encode
    :param mode: number of significant bits
    :return: 
    """

    result = ""

    msg_b = msg.encode("utf8")
    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    bit_seq = []
    for byte in msg_b:  # get bytes to binary values; every bits store to sublist
        bit_seq += list(map(int, f"{byte:08b}"))

    res_len = ceil((len(msg_b) * 8) / mode)     # length of result (bytes)
    bit_seq += [0] * (res_len * mode - len(bit_seq))    # filling zeros

    to_hamming = []

    for i in range(res_len):    # insert control bits into specified positions
        code = bit_seq[i * mode:i * mode + mode]
        for j in range(s_num):
            code.insert(2 ** j - 1, 0)
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, True)   # process

    for i in to_hamming:
        result += "".join(map(str, i))

    return result

In [5]:
def hamming_decode(msg: str, mode: int = 8):
    """
    Decoding the message with Hamming code.
    :param msg: Message string to decode
    :param mode: number of significant bits
    :return: 
    """

    result = ""

    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    res_len = len(msg) // (mode + s_num)    # length of result (bytes)
    code_len = mode + s_num     # length of one code sequence

    to_hamming = []

    for i in range(res_len):    # convert binary-like string to int-list
        code = list(map(int, msg[i * code_len:i * code_len + code_len]))
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, False)  # process

    for i in to_hamming:    # delete control bits
        for j in range(s_num):
            i.pop(2 ** j - 1 - j)
        result += "".join(map(str, i))

    msg_l = []

    for i in range(len(result) // 8):   # convert from binary-sring value to integer
        val = "".join(result[i * 8:i * 8 + 8])
        msg_l.append(int(val, 2))

    # finally decode to a regular string
    try:
        result = bytes(msg_l).decode("utf-8")
    except UnicodeDecodeError:
        pass

    return result, errors

In [6]:
def noizer(msg: str, mode: int) -> str:
    """
    Generates an error in each element of a Hamming encoded message
    """
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize = randrange(code_len)
        to_noize[noize] = int(not to_noize[noize])
        result += "".join(map(str, to_noize))

    return result

In [7]:
def noizer4(msg: str, mode: int) -> str:
    """
    Generates up to 4 errors in each element of a Hamming encoded message
    """
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize1 = randrange(code_len)
        noize2 = randrange(code_len)
        noize3 = randrange(code_len)
        noize4 = randrange(code_len)
        to_noize[noize1] = int(not to_noize[noize1])
        to_noize[noize2] = int(not to_noize[noize2])
        to_noize[noize3] = int(not to_noize[noize3])
        to_noize[noize4] = int(not to_noize[noize4])
        result += "".join(map(str, to_noize))

    return result

In [11]:
MODE = 82  # длина слова с контрольными битами составляет 89 => значащих битов в слове 82
msg = "Imagine Dragons — поп-рок-группа, в состав которой входят четыре человека.\n Была основана в Лас-Вегасе в 2008 году. Название группы представляет собой анаграмму, значение которой знают лишь участники самой группы. История группы началась с того, что Дэниэл Рейнольдс привлёк для участия в собственной группе своего друга и товарища по колледжу барабанщика Эндрю Толмана, который, в свою очередь пригласил в группу свою жену Бриттани Толман (клавишные и бэк-вокал) и Дэниела (Уэйна) Сермона, который отлично играл на гитаре. Группа быстро набирала популярность. А уже в Лас-Вегасе к Дэну и Уэйну присоединился басист Бенджамин (Бен) МакКи, приглашённый Уэйном.\n В 2010 году Эндрю и Бриттани сообщили об уходе из группы для создания семьи. Тогда Бен МакКи пригласил в группу барабанщика Дэниэла (Платца) Платцмана. Позже к ним присоединилась Аврора Флоренс, заменяющая Бриттани, а после её ухода — Тереза Фламинио.\n После ухода Терезы состав группы больше не менялся. В начале 2009 года музыканты начали работать в студии. И уже 1 сентября выпустили свой первый альбом Imagine Dragons. А 1 июня следующего года свет увидел их мини-альбом Hell and Silence (в записи участвовал номинант «Грэмми» Марк Нидхэм). Оба альбома были записаны в звукозаписывающей студии Battle Born Studios. Вскоре им было предложено выступить на ежегодном фестивале Bite of Las Vegas, как «Самая востребованная группа 2010» на местном 107.9FM.\n Они были выбраны хедлайнерами на фестивале Vegas Music Summit, а газета Las Vegas CityLife  (англ.)рус. назвала их «группой, которую необходимо увидеть вживую». По мнению газеты Las Vegas Weekly  (англ.)рус., они стали «Лучшей инди-группой 2010 года», а также были удостоены премии «Лучшая пластинка 2011 года» от журнала Vegas Seven. В ноябре 2011 года Imagine Dragons подписали контракт с американским звукозаписывающим лейблом Interscope Records. Они тесно сотрудничали с продюсером Алексом Да Кидом, с которым они и записали свой первый диск в Уэст-Голливуде.\n Их EP был назван Continued Silence и выпущен 14 февраля 2012 года. 4 сентября 2012 года вышел первый полноценный альбом группы — Night Visions. Пластинка долгое время была в середине списка Billboard-200, лишь к началу 2013 года работа обрела популярность. В частности, хит «Radioactive» был в аутсайдерах списка на момент своего выхода (март 2012 года), однако к апрелю 2013 он достиг 7 места в Billboard Hot 100, 1 места в Billboard Alternative Songs и Billboard Rock Songs.\n Был поставлен абсолютный рекорд за всю более чем полувековую историю по наибольшему времени нахождения в общенациональном хит-параде США: 87 недель в Billboard Hot 100 (май 2014). Сингл вошёл в топ-3 самых продаваемых песен по итогам 2013 года. В итоге композиция получила более 30 платиновых сертификаций и 2 бриллиантовые, в том числе в США."
print(f'Сообщение:\n{msg}')
checksum = crc64(msg)
print(f'Контрольная сумма: {checksum}')

# Первая отправка (без ошибок)
print('-----------ПЕРВАЯ ОТПРАВКА-----------')
enc_msg = hamming_encode(msg, MODE)
print(f'Кодированное сообщение:\n{enc_msg}')
dec_msg, err = hamming_decode(enc_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc64(dec_msg)}, корректность: {crc64(dec_msg) == checksum}')
print(f'MSG: {msg == dec_msg}')

# Вторая отправка (не более 1 ошибки на слово)
print('-----------ВТОРАЯ ОТПРАВКА-----------')
noize_msg = noizer(enc_msg, MODE)
print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
dec_msg, err = hamming_decode(noize_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc64(dec_msg)}, корректность: {crc64(dec_msg) == checksum}')
print(f'MSG: {msg == dec_msg}')

# Третья отправка (4 ошибки на слово)
print('-----------ТРЕТЬЯ ОТПРАВКА-----------')
noize_msg = noizer4(enc_msg, MODE)
print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
dec_msg, err = hamming_decode(noize_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc64(dec_msg)}, корректность: {crc64(dec_msg) == checksum}, количество обнаруженных ошибок: {err}')

Сообщение:
Imagine Dragons — поп-рок-группа, в состав которой входят четыре человека.
 Была основана в Лас-Вегасе в 2008 году. Название группы представляет собой анаграмму, значение которой знают лишь участники самой группы. История группы началась с того, что Дэниэл Рейнольдс привлёк для участия в собственной группе своего друга и товарища по колледжу барабанщика Эндрю Толмана, который, в свою очередь пригласил в группу свою жену Бриттани Толман (клавишные и бэк-вокал) и Дэниела (Уэйна) Сермона, который отлично играл на гитаре. Группа быстро набирала популярность. А уже в Лас-Вегасе к Дэну и Уэйну присоединился басист Бенджамин (Бен) МакКи, приглашённый Уэйном.
 В 2010 году Эндрю и Бриттани сообщили об уходе из группы для создания семьи. Тогда Бен МакКи пригласил в группу барабанщика Дэниэла (Платца) Платцмана. Позже к ним присоединилась Аврора Флоренс, заменяющая Бриттани, а после её ухода — Тереза Фламинио.
 После ухода Терезы состав группы больше не менялся. В начале 2009 года музы